In [7]:
import nltk
import random
from nltk.corpus import movie_reviews
import pickle

In [2]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)
print(documents[0])
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())
# converts word frequency dictionary
all_words = nltk.FreqDist(all_words)
# print(all_words.most_common(15))
# print(all_words["stupid"])
word_features = list(all_words.keys()) # keywords frequency
print(word_features[:100])

(['although', 'i', 'had', 'not', 'been', 'a', 'viewer', 'of', 'the', '"', 'rugrats', '"', 'television', 'series', ',', 'i', 'went', 'into', 'their', 'first', 'animated', 'feature', 'film', ',', '"', 'the', 'rugrats', 'movie', ',', '"', 'with', 'a', 'positive', 'attitude', '.', 'the', 'trailer', 'looked', 'cute', 'enough', ',', 'after', 'all', '.', 'after', 'seeing', 'it', ',', 'i', 'think', 'the', 'words', 'in', 'my', 'recent', '"', 'antz', '"', 'review', ',', 'in', 'which', 'i', 'stated', 'that', 'it', 'was', 'the', 'worst', 'film', 'of', 'its', 'type', 'since', '1995', "'", 's', '"', 'the', 'pebble', 'and', 'the', 'penguin', ',', '"', 'were', 'a', 'bit', 'premature', '.', '"', 'the', 'rugrats', 'movie', ',', '"', 'is', 'bottom', '-', 'of', '-', 'the', '-', 'barrel', 'children', "'", 's', 'fare', 'at', 'its', 'worst', ',', 'and', 'starts', 'to', 'make', ',', '"', 'antz', '"', 'look', 'good', 'in', 'comparison', '.', 'as', 'in', 'the', 'show', ',', '"', 'the', 'rugrats', 'movie', ',', 

In [3]:
# map the keyword to each doc, if the doc's word is the keyword, return true
def find_features(document, word_features):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [4]:
# searching out all feature in documents
features = [(find_features(rev, word_features), category) for (rev, category) in documents]
print(features[0])

({'plot': False, ':': True, 'two': True, 'teen': False, 'couples': False, 'go': True, 'to': True, 'a': True, 'church': False, 'party': False, ',': True, 'drink': False, 'and': True, 'then': False, 'drive': False, '.': True, 'they': True, 'get': True, 'into': True, 'an': True, 'accident': False, 'one': True, 'of': True, 'the': True, 'guys': False, 'dies': False, 'but': True, 'his': True, 'girlfriend': False, 'continues': False, 'see': True, 'him': False, 'in': True, 'her': False, 'life': False, 'has': True, 'nightmares': False, 'what': True, "'": True, 's': True, 'deal': True, '?': True, 'watch': False, 'movie': True, '"': True, 'sorta': False, 'find': False, 'out': False, 'critique': False, 'mind': False, '-': True, 'fuck': False, 'for': True, 'generation': False, 'that': True, 'touches': False, 'on': True, 'very': True, 'cool': False, 'idea': True, 'presents': False, 'it': True, 'bad': False, 'package': False, 'which': True, 'is': True, 'makes': False, 'this': True, 'review': True, 'e

In [5]:
# slip features into train and test sets
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(features, test_size=0.2)
display(len(train_set), len(test_set))

1600

400

## Naive Bayes Classifier
supervised ML, to tell data is negative or positive. In this case, we gonna classify each keyword which is True in each document, that is the positive or negative in features 

In [12]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
test_acc = nltk.classify.accuracy(classifier, test_set)
print(f'Testing accuracy: {test_acc*100}')

Testing accuracy: 78.75


In [13]:
classifier.show_most_informative_features(30)

Most Informative Features
             outstanding = True              pos : neg    =     21.5 : 1.0
               ludicrous = True              neg : pos    =     14.3 : 1.0
                 idiotic = True              neg : pos    =     12.6 : 1.0
                  avoids = True              pos : neg    =     11.3 : 1.0
                  regard = True              pos : neg    =     11.3 : 1.0
              schumacher = True              neg : pos    =     10.7 : 1.0
              astounding = True              pos : neg    =     10.0 : 1.0
               depiction = True              pos : neg    =     10.0 : 1.0
             fascination = True              pos : neg    =     10.0 : 1.0
                    slip = True              pos : neg    =     10.0 : 1.0
                    3000 = True              neg : pos    =     10.0 : 1.0
                     dud = True              neg : pos    =     10.0 : 1.0
                     obi = True              pos : neg    =      9.4 : 1.0

the pos:neg or neg:pos ratio for the word means the word appears more times in positive aspect or negative aspect than in negative or positive.  

In [15]:

# save classifier to disk
save_classifier = open('naivebayes.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [8]:
# load classifier
file = open('naivebayes.pickle', 'rb')
classifier = pickle.load(file)
file.close()

In [9]:
# Scikit-learn Sklearn with NLTK
from nltk.classify.scikitlearn import SklearnClassifier # API nltk classifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [10]:
# combined classifier and other algorithms for purpose.
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(train_set)
print(f'MultinomialNB accuracy= {nltk.classify.accuracy(MNB_classifier, test_set)}')

BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(train_set)
print(f'BernoulliNB accuracy= {nltk.classify.accuracy(BNB_classifier, test_set)}')

MultinomialNB accuracy= 0.8625
BernoulliNB accuracy= 0.8175


In [11]:
# original Naive Bayes classifier
print(f'Naive Bayes acc= {nltk.classify.accuracy(classifier, test_set)}')

Naive Bayes acc= 0.965


In [12]:
#
LogisticReg_classifier = SklearnClassifier(LogisticRegression())
LogisticReg_classifier.train(train_set)
print(f'LogisticRegression acc= {nltk.classify.accuracy(LogisticReg_classifier, test_set)}')

/home/chuongnet/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression acc= 0.8825


In [13]:
LogisticReg_classifier.show_most_informative_features(15)

AttributeError: 'SklearnClassifier' object has no attribute 'show_most_informative_features'

## Combining Algorithms with NLTK

In [2]:
from nltk.classify import ClassifierI
from statistics import mode

In [3]:
# customize the classifier with the ClassifierI
class VoteClassifier(ClassifierI):
    
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
    
    
# voted_classifier = VoteClassifier(classifier, 
#                                  MNB_classifier,
#                                  BNB_classifier,
#                                  LogisticReg_classifier)

# print(f'Voted classifier acc= {(nltk.classify.accuracy(voted_classifier, test_set)) * 100}')

# print('classification: ', voted_classifier.classify(test_set[0][0]), 
#       ', confidence= ', voted_classifier.confidence(test_set[0][0]))
# print('classification: ', voted_classifier.classify(test_set[1][0]), 
#       ', confidence= ', voted_classifier.confidence(test_set[1][0]))
# print('classification: ', voted_classifier.classify(test_set[2][0]), 
#       ', confidence= ', voted_classifier.confidence(test_set[2][0]))
# print('classification: ', voted_classifier.classify(test_set[3][0]), 
#       ', confidence= ', voted_classifier.confidence(test_set[3][0]))
